# Loading the data

In [2]:
%matplotlib inline

import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import numpy as np

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer

/root/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
train = pd.read_table('train.tsv')
test = pd.read_table('test.tsv')


In [4]:
train.head()


,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [5]:
test.head()

,PhraseId,SentenceId,Phrase
0,156061,8545,An intermittently pleasing but mostly routine ...
1,156062,8545,An intermittently pleasing but mostly routine ...
2,156063,8545,An
3,156064,8545,intermittently pleasing but mostly routine effort
4,156065,8545,intermittently pleasing but mostly routine


In [6]:
Phrase = train['Phrase']
Sen = train['Sentiment']
Phrase2 = test['Phrase']

In [7]:
print(train.head(20))

    PhraseId  SentenceId                                             Phrase  \
0          1           1  A series of escapades demonstrating the adage ...   
1          2           1  A series of escapades demonstrating the adage ...   
2          3           1                                           A series   
3          4           1                                                  A   
4          5           1                                             series   
5          6           1  of escapades demonstrating the adage that what...   
6          7           1                                                 of   
7          8           1  escapades demonstrating the adage that what is...   
8          9           1                                          escapades   
9         10           1  demonstrating the adage that what is good for ...   
10        11           1                            demonstrating the adage   
11        12           1                            

In [8]:
tmp = train


In [9]:
tmp1 = test

In [10]:
review_list=[]
for review in tmp['Phrase']:
    review=review.lower()
    review_list.append(review)
    #print(review)
    

In [11]:
review_list1=[]
for review1 in tmp1['Phrase']:
    review1=review1.lower()
    review_list1.append(review1)
    #print(review)
    

In [12]:
tmp=tmp.drop('Phrase',axis=1)

In [13]:
tmp1=tmp1.drop('Phrase',axis=1)

In [14]:
tmp['Phrase']=review_list

In [15]:
tmp1['Phrase']=review_list1

In [16]:
tmp.head(5)

,PhraseId,SentenceId,Sentiment,Phrase
0,1,1,1,a series of escapades demonstrating the adage ...
1,2,1,2,a series of escapades demonstrating the adage ...
2,3,1,2,a series
3,4,1,2,a
4,5,1,2,series


In [17]:
tmp1.head()

,PhraseId,SentenceId,Phrase
0,156061,8545,an intermittently pleasing but mostly routine ...
1,156062,8545,an intermittently pleasing but mostly routine ...
2,156063,8545,an
3,156064,8545,intermittently pleasing but mostly routine effort
4,156065,8545,intermittently pleasing but mostly routine


# Cleaning the data

To format our data and build the Term-doc incidence matrix, many operations will be performed on the data :

    * Stemming
    * Stop words removal
    * Lowering
    * Tokenization
    * Pruning (numbers and punctuation)



In [18]:
from nltk.corpus import stopwords
from string import punctuation
punc = set(punctuation)
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
from nltk.tokenize import wordpunct_tokenize

In [19]:
tokenized = []
for review in tmp['Phrase']:
    
    tokens = nltk.word_tokenize(review)
    
    tokenized.append(tokens)

In [22]:
tokenized1 = []
for review1 in tmp1['Phrase']:
    
    tokens1 = nltk.word_tokenize(review1)
    
    tokenized1.append(tokens1)

In [25]:
tokenized[1:5]

[['a',
  'series',
  'of',
  'escapades',
  'demonstrating',
  'the',
  'adage',
  'that',
  'what',
  'is',
  'good',
  'for',
  'the',
  'goose'],
 ['a', 'series'],
 ['a'],
 ['series']]

In [26]:
tokenized1[1:5]

[['an', 'intermittently', 'pleasing', 'but', 'mostly', 'routine', 'effort'],
 ['an'],
 ['intermittently', 'pleasing', 'but', 'mostly', 'routine', 'effort'],
 ['intermittently', 'pleasing', 'but', 'mostly', 'routine']]

In [27]:
stopwords_json = {"en":["a","a's","able","about","above","according","accordingly","across","actually","after","afterwards","again","against","ain't","all","allow","allows","almost","alone","along","already","also","although","always","am","among","amongst","an","and","another","any","anybody","anyhow","anyone","anything","anyway","anyways","anywhere","apart","appear","appreciate","appropriate","are","aren't","around","as","aside","ask","asking","associated","at","available","away","awfully","b","be","became","because","become","becomes","becoming","been","before","beforehand","behind","being","believe","below","beside","besides","best","better","between","beyond","both","brief","but","by","c","c'mon","c's","came","can","can't","cannot","cant","cause","causes","certain","certainly","changes","clearly","co","com","come","comes","concerning","consequently","consider","considering","contain","containing","contains","corresponding","could","couldn't","course","currently","d","definitely","described","despite","did","didn't","different","do","does","doesn't","doing","don't","done","down","downwards","during","e","each","edu","eg","eight","either","else","elsewhere","enough","entirely","especially","et","etc","even","ever","every","everybody","everyone","everything","everywhere","ex","exactly","example","except","f","far","few","fifth","first","five","followed","following","follows","for","former","formerly","forth","four","from","further","furthermore","g","get","gets","getting","given","gives","go","goes","going","gone","got","gotten","greetings","h","had","hadn't","happens","hardly","has","hasn't","have","haven't","having","he","he's","hello","help","hence","her","here","here's","hereafter","hereby","herein","hereupon","hers","herself","hi","him","himself","his","hither","hopefully","how","howbeit","however","i","i'd","i'll","i'm","i've","ie","if","ignored","immediate","in","inasmuch","inc","indeed","indicate","indicated","indicates","inner","insofar","instead","into","inward","is","isn't","it","it'd","it'll","it's","its","itself","j","just","k","keep","keeps","kept","know","known","knows","l","last","lately","later","latter","latterly","least","less","lest","let","let's","like","liked","likely","little","look","looking","looks","ltd","m","mainly","many","may","maybe","me","mean","meanwhile","merely","might","more","moreover","most","mostly","much","must","my","myself","n","name","namely","nd","near","nearly","necessary","need","needs","neither","never","nevertheless","new","next","nine","no","nobody","non","none","noone","nor","normally","not","nothing","novel","now","nowhere","o","obviously","of","off","often","oh","ok","okay","old","on","once","one","ones","only","onto","or","other","others","otherwise","ought","our","ours","ourselves","out","outside","over","overall","own","p","particular","particularly","per","perhaps","placed","please","plus","possible","presumably","probably","provides","q","que","quite","qv","r","rather","rd","re","really","reasonably","regarding","regardless","regards","relatively","respectively","right","s","said","same","saw","say","saying","says","second","secondly","see","seeing","seem","seemed","seeming","seems","seen","self","selves","sensible","sent","serious","seriously","seven","several","shall","she","should","shouldn't","since","six","so","some","somebody","somehow","someone","something","sometime","sometimes","somewhat","somewhere","soon","sorry","specified","specify","specifying","still","sub","such","sup","sure","t","t's","take","taken","tell","tends","th","than","thank","thanks","thanx","that","that's","thats","the","their","theirs","them","themselves","then","thence","there","there's","thereafter","thereby","therefore","therein","theres","thereupon","these","they","they'd","they'll","they're","they've","think","third","this","thorough","thoroughly","those","though","three","through","throughout","thru","thus","to","together","too","took","toward","towards","tried","tries","truly","try","trying","twice","two","u","un","under","unfortunately","unless","unlikely","until","unto","up","upon","us","use","used","useful","uses","using","usually","uucp","v","value","various","very","via","viz","vs","w","want","wants","was","wasn't","way","we","we'd","we'll","we're","we've","welcome","well","went","were","weren't","what","what's","whatever","when","whence","whenever","where","where's","whereafter","whereas","whereby","wherein","whereupon","wherever","whether","which","while","whither","who","who's","whoever","whole","whom","whose","why","will","willing","wish","with","within","without","won't","wonder","would","wouldn't","x","y","yes","yet","you","you'd","you'll","you're","you've","your","yours","yourself","yourselves","z","zero"]}


In [28]:
stopwords_json_en = set(stopwords_json['en'])
stopwords_nltk_en = set(stopwords.words('english'))
stopwords_punct = set(punctuation)
spe = ["''","``"]
spe = set(spe)
# Combine the stopwords. Its a lot longer so I'm not printing it out...
stop_pun = set.union(stopwords_json_en, stopwords_nltk_en, stopwords_punct,spe)

In [29]:
s = []
for ele in tokenized:
    for i in ele:
        if i in stop_pun:
            ele.remove(i)
        if type(i)==type(5) or type(i)==type(5.0):
            print(i)
            ele.remove(i)
            
            


In [30]:
tmp['Phrase'] = tokenized

In [31]:
tmp.head()

,PhraseId,SentenceId,Sentiment,Phrase
0,1,1,1,"[series, escapades, demonstrating, adage, what..."
1,2,1,2,"[series, escapades, demonstrating, adage, what..."
2,3,1,2,[series]
3,4,1,2,[]
4,5,1,2,[series]


In [32]:
s1 = []
for ele in tokenized1:
    for i in ele:
        if i in stop_pun:
            ele.remove(i)
        if type(i)==type(5) or type(i)==type(5.0):
            print(i)
            ele.remove(i)

In [33]:
tmp1['Phrase'] = tokenized1

In [34]:
tmp1.head()

,PhraseId,SentenceId,Phrase
0,156061,8545,"[intermittently, pleasing, mostly, routine, ef..."
1,156062,8545,"[intermittently, pleasing, mostly, routine, ef..."
2,156063,8545,[]
3,156064,8545,"[intermittently, pleasing, mostly, routine, ef..."
4,156065,8545,"[intermittently, pleasing, mostly, routine]"


In [35]:
#X_train, X_test, y_train, y_test = train_test_split(Phrase,Sentiment, test_size=0.2, random_state=42)

In [50]:
def list_to_str(x):
    y=str(x)
    y1=y.replace('[','')
    y2=y1.replace(']','')
    y3=y2.replace(",","")
    y4=y3.replace("'","")
    return y4


In [51]:
tmp['Phrase1']=tmp[['Phrase']].applymap(lambda x:list_to_str(x))


In [53]:
tmp1['Phrase1']=tmp1[['Phrase']].applymap(lambda x:list_to_str(x))


In [54]:
X_train = tmp['Phrase1']
Y_train = Sen

In [58]:
X_test2 = tmp1['Phrase1']

In [59]:
count_vect = CountVectorizer()
X_train_final = count_vect.fit_transform(X_train)  
X_test2_final = count_vect.transform(X_test2)


tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_final)
X_test2_tfidf = tfidf_transformer.fit_transform(X_test2_final)



# Applying Multinomial Naïve Bayes learning method¶

In [60]:
prediction = dict()
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB().fit(X_train_tfidf, Y_train)
prediction['Multinomial'] = model.predict(X_test2_tfidf)

In [61]:
len(prediction['Multinomial'])

66292

# Before we test our classifier on the test set, we get a sense of how good it is on the validation set.

In [62]:
#from sklearn.metrics import accuracy_score

#print('Accuracy = {}'.format(
 #       accuracy_score(prediction['Multinomial'], y_test) * 100)
  #   )
#Accuracy = 93.63300442603646


In [63]:
subdf = pd.read_csv('sampleSubmission.csv')

In [64]:
subdf.head()

,PhraseId,Sentiment
0,156061,2
1,156062,2
2,156063,2
3,156064,2
4,156065,2


In [65]:
len(tmp1['PhraseId'])

66292

In [66]:
Phrase = tmp1['PhraseId']
Sent= prediction['Multinomial']

In [67]:
Sent = list(Sent)

In [68]:
st=pd.DataFrame({'Sentiment':Sent})

In [69]:
predicted=pd.concat([Phrase,st],axis=1)

In [70]:
predicted.head()

,PhraseId,Sentiment
0,156061,3
1,156062,3
2,156063,2
3,156064,3
4,156065,3


In [71]:
predicted.to_csv('submission.csv',index=False)